# Jupyter Notebook Testing

Jupyter Notebooks use cells that can easily be re ran and is very convenient for testing things out.
It's important to note that to be able to run these cells correctly, the dependencies need to be properly installed.
This means that the interpreter being used must point to the Python the one that is in your [venv](./.venv) created by poetry for this project.

**Note**: if you make changes to the source code, you will need to press `Restart` above to restart the kernel to ensure the changes occur here.
![image](../pictures/restartJupyterKernel.png)

In [2]:
import logging

logging.basicConfig(level=logging.INFO)
from engine.types.industry_type import IndustryType
from engine.types.demographic import Demographic


policies = {
    "corporate_income_tax": {itype.value: 0.0 for itype in IndustryType},
    "personal_income_tax": {demo.value: 3 for demo in Demographic},
    "sales_tax": {itype.value: 0.0 for itype in IndustryType},
    "property_tax": 0.0,
    "tariffs": {itype.value: 0.0 for itype in IndustryType},
    "subsidies": {itype.value: 0.0 for itype in IndustryType},
    "rent_cap": 0.0,
    "minimum_wage": 0.0,
}

industries = {
    itype: {
        "price": 10.0 + i * 5.0,
        "inventory": 1000 + i * 500,
        "balance": 50000 + i * 10000,
        "offered_wage": 15.0 + i * 2.5,
    }
    for i, itype in enumerate(IndustryType)
}

demographics = {
    demo: {
        "income": {"mean": 300 + (i * 500), "sd": 10 + (i * 100)},
        "proportion": 1 / len(Demographic),
        "unemployment_rate": 0.1 - (i * 0.02),
        "spending_behavior": {
            itype.value: 1 / len(IndustryType) for itype in IndustryType
        },
        "balance": {"mean": 500 + (i * 2000), "sd": 100},
    }
    for i, demo in enumerate(Demographic)
}

In [3]:
from engine.interface.controller import ModelController
from api.city_template import CityTemplate

config = CityTemplate.SMALL.config
controller = ModelController()
model_id = controller.create_model(
    max_simulation_length=100,
    num_people=100,
    demographics=config["demographics"],
    industries=config["industries"],
    starting_policies=config["policies"],
    inflation_rate=config["inflation_rate"],
)

In [4]:
controller.step_model(model_id,time=1)

INFO:MESA.mesa.model:calling model.step for timestep 1 
INFO:root:Changing employment...NOT IMPLEMENTED
INFO:root:Changing employment...NOT IMPLEMENTED
INFO:root:Changing employment...NOT IMPLEMENTED
INFO:root:Changing employment...NOT IMPLEMENTED
INFO:root:Changing employment...NOT IMPLEMENTED
INFO:root:Changing employment...NOT IMPLEMENTED
INFO:root:Changing employment...NOT IMPLEMENTED
INFO:root:Changing employment...NOT IMPLEMENTED
INFO:root:Changing employment...NOT IMPLEMENTED
INFO:root:Changing employment...NOT IMPLEMENTED
INFO:root:Changing employment...NOT IMPLEMENTED
INFO:root:Changing employment...NOT IMPLEMENTED
INFO:root:Changing employment...NOT IMPLEMENTED


In [5]:
metrics = controller.get_demo_metrics(model_id)

metrics

,week,Demographics,Proportion,Average Balance,Balance Std,Average Wage,Wage Std
0,1,Lower Class,0.35,12.152710,0.642131,7.257967,0.381890
1,1,Middle Class,0.50,15.525719,0.511226,8.048030,0.236817
2,1,Upper Class,0.15,18.627923,0.885879,8.803380,0.574066
